In [1]:
import codecs
import collections
import requests
import sys
#  import utils
import datetime
from pymystem3 import Mystem
import re
import pandas as pd
import html2text

replace_list = ["|","?",".", "!" ,",", "\"","\\", "-","/", ":", ";",  "â€œ", "â€˜", "*", ">", "—", """[""", "]", "_", "+",  "✔",')','('] 



def replace_other(text):
    """This function replace all not necessary symbols in the text"""
    for i in replace_list:
        text = text.replace(i, " ")

    return (text)

def handle_text(text):
    """This function handle text and make it in the good format"""
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.escape_all = True
    text = h.handle(text)
    text = replace_other(text)
    return (text)

def analyze_text(text):
    RU = Mystem()
    lemmas = RU.lemmatize(text)
    analysis = RU.analyze(text)
    type_pattern = re.compile("^([A-Z]+)*")
    
    while (' ' in lemmas) == True: # очистили от пробелов
        lemmas.remove(' ')

    while ({'text': ' '} in analysis) == True: # очистили от пробелов
        analysis.remove({'text': ' '})

    while ({'text': '\n'} in analysis) == True: # очистили от \n
        analysis.remove({'text': '\n'})
    return (lemmas, analysis)

In [46]:
# df["Full text"] = df["Full text"].astype(str)
text = """
я так понимаю, у меня не все хорошо. Сказали, дочу не таскать, но как, я пока не представляю...мы на гв и по ночам приходится на руках качать.
Врач узи прописала дюфастон по 1 таблетки 2 раза в день, а может ещё чего попить? Гематома мне кажется большая. ...


""" 
# df["Full text"].str.cat(sep=' ')
# text [:1000]

text = handle_text(text)
text = re.sub('[a-z]', ' ', text)
text = re.sub('[A-Z]', ' ', text)
text = re.sub('[0-9]', ' ', text)
text = text.replace("\n!", " ")
text = text.replace("\n", " ")
print (text[:1000])

# в функцию необходимо добавить фильтр по длине слов (нам нужно >3)

я так понимаю  у меня не все хорошо  Сказали  дочу не таскать  но как  я пока не представляю   мы на гв и по ночам приходится на руках качать  Врач узи прописала дюфастон по   таблетки   раза в день  а может ещё чего попить  Гематома мне кажется большая       


In [47]:
list_text = text.split()
lemmas = []
analysis = []
count = 0

print (len(list_text))

while count < len(list_text):  # скармиливаю по чуть-чуть, чтобы комп справлялся
    text_new = ' '.join(list_text [count:count+500])
    count += 500
    print (count/len(list_text)*100)
    lemmas_1, analysis_1 = analyze_text (text_new)
    lemmas = lemmas + lemmas_1
    analysis = analysis + analysis_1
    

text_new = ' '.join(list_text [count:len(list_text)])
lemmas_1, analysis_1 = analyze_text (text_new)
lemmas = lemmas + lemmas_1
analysis = analysis + analysis_1

print ('It has done')

46
1086.9565217391305
It has done


In [48]:
list_right_words = []
list_right_words_1 = []
stop_words = ["уже","ещё","затем","еще","также","раз"]
for i in range(len(analysis)):
    if ((str(analysis [i])[2:6]) != 'text' and ((not((analysis [i].get("analysis")))) == False)):
#             print (str(analysis [i]))
#             print ( (not((analysis [i].get("analysis")))) == False)
            s = (analysis [i].get("analysis")[0].get('gr'))
            part_speech = s.replace(',', ' ').split()[0]
            if '=' in part_speech:
                part_speech = part_speech [:part_speech.find("=")]
            if (part_speech == 'S'  or part_speech == 'A' or part_speech == 'V'): #or part_speech == 'A' or part_speech == 'ADV'    due to Nataly or part_speech == 'V'
                list_right_words.append (((analysis [i]).get("analysis")[0]).get('lex'))
#                 print (part_speech, '----',((analysis [i]).get("analysis")[0]).get('lex')) # ADV ---- часто
#             print (s)
for i in list_right_words:
    # ниже необходимо переписать в функцию и наверх !!!!
    if i not in stop_words:   
        list_right_words_1.append (i)
# my_file = open('BB_cloud' + '_pre' + '.txt', 'a', encoding='utf-16')
# my_file.write(text)
# my_file.close()

In [49]:
print (list_right_words_1)

['понимать', 'сказать', 'доча', 'таскать', 'представлять', 'ночь', 'приходиться', 'рука', 'качать', 'врач', 'узи', 'прописывать', 'дюфастон', 'таблетка', 'день', 'мочь', 'попить', 'гематома', 'казаться', 'больший']


In [105]:
from py_translator import Translator

In [107]:
translator = Translator() 
s = translator.translate(text='"ДД, ничего', dest='en').text
print(s)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [115]:
from yandex_translate import YandexTranslate
translate = YandexTranslate('trnsl.1.1.20181211T161237Z.7437d1e24da768b7.8f59bb17533affb11f498c68af8aa3bfb2d0a088')
# print('Languages:', translate.langs)
# print('Translate directions:', translate.directions)
# print('Detect language:', translate.detect('Привет, мир!'))
print('Translate:', translate.translate('Привет, мир!', 'ru-en'))  # or just 'en'

ImportError: cannot import name 'Translater' from 'yandex_translate' (C:\ProgramData\Anaconda3\lib\site-packages\yandex_translate\__init__.py)

Languages: {'no', 'lv', 'sq', 'pt', 'da', 'hy', 'sv', 'hu', 'mk', 'nl', 'tr', 'sl', 'az', 'es', 'ru', 'pl', 'uk', 'el', 'it', 'be', 'fr', 'hr', 'lt', 'fi', 'bg', 'cs', 'ro', 'ca', 'sk', 'sr', 'et', 'de', 'en'}
Translate directions: ['az-ru', 'be-bg', 'be-cs', 'be-de', 'be-en', 'be-es', 'be-fr', 'be-it', 'be-pl', 'be-ro', 'be-ru', 'be-sr', 'be-tr', 'bg-be', 'bg-ru', 'bg-uk', 'ca-en', 'ca-ru', 'cs-be', 'cs-en', 'cs-ru', 'cs-uk', 'da-en', 'da-ru', 'de-be', 'de-en', 'de-es', 'de-fr', 'de-it', 'de-ru', 'de-tr', 'de-uk', 'el-en', 'el-ru', 'en-be', 'en-ca', 'en-cs', 'en-da', 'en-de', 'en-el', 'en-es', 'en-et', 'en-fi', 'en-fr', 'en-hu', 'en-it', 'en-lt', 'en-lv', 'en-mk', 'en-nl', 'en-no', 'en-pt', 'en-ru', 'en-sk', 'en-sl', 'en-sq', 'en-sv', 'en-tr', 'en-uk', 'es-be', 'es-de', 'es-en', 'es-ru', 'es-uk', 'et-en', 'et-ru', 'fi-en', 'fi-ru', 'fr-be', 'fr-de', 'fr-en', 'fr-ru', 'fr-uk', 'hr-ru', 'hu-en', 'hu-ru', 'hy-ru', 'it-be', 'it-de', 'it-en', 'it-ru', 'it-uk', 'lt-en', 'lt-ru', 'lv-en', 'l

In [85]:
# data = pd.read_excel("SQL\\"+"Babyblog_sql.xlsx")

data


,ID_posts,Author_name,Title_post,Post_Preview,Time_of_post,Type_of_post,Href_post
0,2|3417879|2807135|3|2807135,Наталья,"Дюфастон, пить или не пить","Привет девочки! Сегодня делала УЗИ, все отличн...",8 часов назад,post,/community/post/conception/3417879
1,2|3169447|1728677|1049|1728677,Кира,Лекарства,Девочки.привет. Хочу продать все лекарства с э...,9 часов назад,post,/community/post/belg_obyavleniya/3169447
2,2|3224063|2947641|5|2947641,Алена,Отмена дюфастона,"Девочки , лежу в больнице с жутким токсикозом ...",10 часов назад,post,/community/post/living/3224063
3,2|3411905|2736613|3489|2736613,Лёка,Страхи...,"Всем привет! Срок 13-14 недель, вчера появилас...",11 часов назад,post,/community/post/happypregn/3411905
4,2|3213853|2069909|4|2069909,Sascha,Пурегон 300,\n,12 часов назад,post,/community/post/sterility/3213853
5,2|3417843|3019247|3|3019247,Светлана,Дюфастон и БТ,Девочки!информативна ли БТ при приеме дюфа?пер...,13 часов назад,post,/community/post/conception/3417843
6,2|3213847|2976305|4|2976305,Juli,Замена Крайнона,"Привет! Девочки, у кого в поддержке был крайно...",20 часов назад,post,/community/post/sterility/3213847
7,2|3213835|2962853|4|2962853,Катерина,"Отдам Фрагмин 2500 МЕ, сиофор 850, инозитол",Остались лекарства от подготовки к протоколу. ...,23 часа назад,post,/community/post/sterility/3213835
8,1|10803|3048453|3048453|3048453,Анна,Вопрос про дюфастон,"Недавно я писала про свои страхи, когда узнала...",23 часа назад,post,/user/annamaks/10803
9,1|34725|3016911|3016911|3016911,\nЛюбовь\n,Схема отмена утрожестана,None,"21 ноября, 20:13",question,/questions/lyubovsavosina/34725


In [81]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

# STOPWORDS_ru = set(stopwords.words('russian'))
# print (len(list(STOPWORDS_ru))) # 151 слово в русском

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = REPLACE_BY_SPACE_RE.sub(" ", text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub("", text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text_list = text.split(" ")
    text_list = [x for x in text_list if x != '']
    text = ' '.join([word for word in text_list if word not in stopwords.words('english')])# delete stopwords from text
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pekhtdx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
replace_list = ["|","?",".", "!" ,",", "\"","\\", "-","/", ":", ";",  "â€œ", "â€˜", "*", ">", "—", """[""", "]", "_", "+",  "✔",')','('] 



def replace_other(text):
    """This function replace all not necessary symbols in the text"""
    for i in replace_list:
        text = text.replace(i, " ")

    return (text)

def handle_text(text):
    """This function handle text and make it in the good format"""
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.escape_all = True
    text = h.handle(text)
    text = replace_other(text)
    return (text)


In [118]:
post_preview_translate = []

for i in (data["Post_Preview"]):

    text_prepared = handle_text(str(i))
    text_prepared = replace_other(str(text_prepared))
    print (text_prepared)
    s = translate.translate(text_prepared, 'ru-en')
    
#     print (s)
    post_preview_translate.append (s)
print (post_preview_translate)

Привет девочки  Сегодня делала УЗИ  все отлично  цикл прям по часам и О тоже
по плану  но меня растроило что эндометрий 6мм  врач сказала маловат и
назначила дюфастон с 11 дня по 25  Кто пил его так же  И помогает ли он  Я
просто где то прочитала  что в    


Девочки привет  Хочу продать все лекарства с эко 


Девочки   лежу в больнице с жутким токсикозом   последнии дни чувствую себя
лучше   дюфастон до этого пила 2 таблетки в день   последнии 3 дня по одной  
сегодня врач сказала вообще отменять   а у меня только 14 недель   что думаете
  Отменят   


Всем привет  Срок 13 14 недель  вчера появилась мазня коричневого цвета  после
полового акта и поднятии ребенка  Съездила на узи   сердечко бьется все в
порядке   сказали что поднимается  детское место  и поэтому мажет  В
заключении написали ретрохориаль   





Девочки информативна ли БТ при приеме дюфа первый цикл принимаю его до этого
цикла мерила БТ циклы правда ановуляторные были  но все же сегодня последняя
таблетка дюфа  с каждым

На сроке 7 2 сдавала гормоны   которые рекомендовала мониторить репродуктолог
и когда получила результаты очень распереживалась   т  К  Прогестерон 39 7
нмоль л 😣  в 1 дз был 65 нмоль л    хгч соответствовал сроку   122000  
 Сегодня по узи все хорошо 🤩 


Девочки привет  Хочу продать все лекарства с эко 


Добрый день  Сегодня 22 дц  я правильно понимаю  что прогестерон в норме  Пью
дюфастон  но гинеколог сказал все равно сдать


Я так надеялась  что спустя 2 года попыток инсеминация нам поможет  Считала
дни после О  записывала свои ощущения  А ведь все действительно было иначе в
этом цикле 


Итак  сегодня поняла  что жду в этом цикле зря  Запишу свои признаки  может 
кому пригодится  3 4ДПО Тянет живот  началась молочница  которой не было год 
если не больше  Грудь молчит  что странно  обычно после О сразу болеть
начинает  Дикая раздражительн   


Девочки  всем привет  Ростов на Дону  Подскажите  где купить подешевле
утрожестан  дюфастон и эстрожель  Хочу сразу закупиться  а то в ап

Прогестерон 38 нмоль   как то не растет вообще со дня подсадки  был на момент
подсадки 33  В нормах на мои недели написано 59 69нмоль  Поддержка 600 утрик и
2р д дюфастон  Что скажете девочки  Ре не назначала  я сама сдала хгч и
прогестерон  Сегодня 19дпп  Касаемо ХГЧ  10 дпп 580 13 дпп 1546 18 дпп 14011
Из самочуствия потихоньку раскачивается токс  тошнота  падает давление в
душных помещениях  из за нападающей тошноты нет аппетита  из за перепадов
настроения гнусно от самой себя  но я счастлива ведь малыш со мной  судя по
хгч🤰🏼 узи аж 6 декабря  но хочу лишний раз лазить туда датчиком  Всем хочушкам
скорее увидеть    а будущим мамочкам легкой беременности и родов 🌷🌷🌷


Уже много времени прошло  но я всё же напишу  потому что никак покоя не даёт 
что другие могут пострадать так же как и я 


Пытались забеременеть много лет  безуспешно  цикл начался в 16лет  35 45дней 
Сейчас мне 30 лет  2 5 года назад поставили диагноз поликистоз бесплодие 1 
Предложили лапару  но 2 года я не решалась 

Есть Утрожестан  Дюфастон  Клексан  Предложения и условия в личку  


Девочки  подскажите пожалуйста  Сегодня сдала хгч и прогестерон  Хгч 7672 
прогестерон 11 7 нм  По лабораторным анализам норма от 9 до 38  Срок по мес
5 6 недель  Знакомой на 6 неделях сказали  что это низкий прогестерон и
назначили дюфастон  Мне пока ни   


Всем привет  Может кто сталкивался с такой же проблемой    эндометрий на 10 
на 14  на 24 дц   вообщем всегда трёхслойный  не переходит во вторую фазу 
овуляция есть  всегда стадия пролиферацией    нет секреции Дюфастон пью   но
она меня равно не меняется   


Всем привет  У меня все также задержка  ни одного намека на эти дни  живот как
тянуло слева так и тянет    Принимаю дюфастон  вычитала в интернете  что он
поможет вызвать эти дни  вот и пытаюсь  с 1 дня задержки   Сегодня уже 4 дз   
Как можно из вызвать   


Девочки заранее спасибо за ответы😘 На 15 16ДЦ была О   тест на 14ДЦ  С 14ДЦ
соски очень болят при касании с 15ДЦ слизистые выделения 16ДЦ водянистые 

Привет  1 й вопрос  участковая выписала по 2 таблетки дюфастона  месяц  и ушла
в отпуск   до этого было 3 таблетки  меня хватило только на 2 недели  а потом
выявился беременный мозг который постоянно забывает пить таблетку вечернюю 
ходила на УЗИ     


Запишу для дневника  Честно говоря  это очень удобно потом читать записи 
Жаль  что в первую беременность я начала писать только с 20 недели  Итак   


Девочки  до какого дня пить шалфей  и с какого дня пить дюфастон  Цикл 26
дней  овуляция предположительно на 12  Спасибо


Девочки  привет  В прошлом цикле удалили полип  гистера с РДВ  в этом цикле по
результатам гистологии назначили Дюфастон во второй фазе  Пила с 15 по 25 ДЦ 
До этого его никогда не пила  26 дц делала тест  клеаблю плюс  отрицательный 
Жду М через сколько   


Г назначил пить дюфастон с 16дц по 25дц при планировании беременности   Пила 2
цикла   цикл 28 дней  все было хорошо   менструация приходила вовремя   через
2 3дня после отмены дюфастона    Сейчас у меня идёт 3 

...

после отмены дюфастона прошло 5 дней а мес нет  на 25 дц сдавала анализы все
мужские гормоны завышены и инсулин немного завышен  у кого после отмены
дюфастона как приходили мес 


5 день отмены дюфастона а мес нет 😩 что это  на 25 дц сдавала гормоны мужские
все завышены  почему может быть задержка 


Девочки  кто на поддержке зачатия Дюфастоном  подскажите  как быть  Сегодня 24
дц  завтра последний день приема  тесты на беременность чистые  но и рано еще
на мой взгляд  так как овуляция была 2 числа  15 дц  По идеи завтра принимаю и
все  отмена до сл   


Привет  девочки  3 5 недели назад удалили полип и сделали конизацию шейки
матки  прокапали курс метранидазола  был воспалительный   Также с 16 по 25 ДЦ
дюфастон  2 месяца   Сейчас шестой день менструации  пришла как по часам  
Вопросик  обычно менструаци   


Привет  девочки  3 5 недели назад удалили полип и сделали конизацию шейки
матки  прокапали курс метранидазола  был воспалительный   Также с 16 по 25 ДЦ
дюфастон  2 месяца   Сейчас




1   Возраст на момент проведения процедуры  мне 32  мужу 35  2   ЭКО либо
ЭКО ИКСИ  ЭКО  часть эмбрионов ЭКО ИКСИ  3   Показания к ЭКО  бесплодие
неясного генеза  3 года безуспешных попыток планирования 


Девочки  что делать  Была задержка 12 дней  Сходила на УЗИ  Функциональная
киста 4 6 на 4 2  Пропила 10 дней дюфастон  Дождалась ещё одних месячных  По
УЗИ она такая же  И что теперь делать с ней  Тянет очень  К врачу только через
10 дней попаду    было    


Девочки ситуация такая  тесты вроде как положительные  последний делала вчера
это по моим подсчетам 13 14 ДПО  но полоска значительно светлее контрольной 
На 11ДПО очень стреляло в яичниках  и болел и живот и поясница  потом начало
мазать бежевым  сходил   


Добрый день В продолжение поста
https   m babyblog ru community post conception 3374649 пересдан пролактин 19
при норме до 23 как так а я уже думада нашли причину моих   


Почти три года у нас не получается зачать ребеночка  Лечилась в платной
клинике  Анализы в норме  

В общем на двух стимуляциях ничего не получилось  Во вторую был хгч 23  через
два дня 6  Отменила дюфастон  пришли М  Врач сказала отдыхать месяца 2 3 
может быть на отмене получится  Вот первый цикл после стимуляции  Сильно болел
яичник на 12 дц  21 02    


Может быть кому то пригодится или кто то слышал об этом  01 03 делали перенос 
из поддержки только утрожестан  Попросила ре заменить его на дюфастон  тк
работаю и свечами по утрам не совсем комфортно пользоваться  Она мне сказала 
что сейчас ведутся исслед   


Продублирую еще и в категории  Итак  что имею из косвенных признаков  возможно
это и мои выдумки   иногда кружится голова  особенно на работе и по утрам 
стала спать и днем  потому что хочется  грудь болит немного  не скажу  что она
увеличилась  списываю это на   


Итак  что имею из косвенных признаков  возможно это и мои выдумки   иногда
кружится голова  особенно на работе и по утрам  стала спать и днем  потому что
хочется  грудь болит немного  не скажу  что она увеличила

Девочки  подскажите  у кого когда пришли месячные после внематочной  Я имею в
виду не кровотечение сразу после операции  а именно месячные  Мне говорили 
что должны придти спустя месяц после лапароскопии  но операцию сделали 25 12 
прошло больше месяца     


Здравствуйте дорогие мои  Я снова пошла в бой  В субботу была подсажена моя
трехдневная бусинка  Впервые крио в ЕЦ  До переноса 3 дня 2 раза в день
утрожестан  сейчас в поддержке 3 р утрожестан  3 р дивигель  3 р дюфастон  1 р
тромбо АСС  фолиевая кислота  До э   


Девочки  привет  Так как сама в ожидании хгч и после первых результатов
постоянно рыскала по страницам сообщества в поисках статистики и хороших
историй  решила поделиться своими цифрами и ощущениями в протоколе  вдруг кому
пригодится  Буду краткой и не    


Не могу сидеть сложа руки после 4 протокола и бхб сразу пошла на узи дабы
провериться и на узи показало что зреет 2 фоллика  я решила попробовать в ец
но не успели пунктировать отслеживали и по узи и по тестам на


YandexTranslateException: ERR_SERVICE_NOT_AVAIBLE

In [ ]:
data ["Post_Preview_en"] = post_preview_traslate
def sentiment_it (string):
#     spisok = string.split ()
    tweet_sentiment_sum = 0
    for i in string:
        if i in text_sentiment_dict:
            tweet_sentiment_sum += int(text_sentiment_dict[i])
    return (tweet_sentiment_sum)

text_sentiment_column = data['Post_Preview_en'].str.split(' ').apply(sentiment_it) 
data["text_sentiment"] = text_sentiment_column

In [70]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

# reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
# X_test = cv.transform(reviews_test_clean)

NameError: name 'reviews_train_clean' is not defined

In [119]:
post_preview_translate

[{'code': 200,
  'lang': 'ru-en',
  'text': ['Hi girls had a scan Today everything is fine cycle just like clockwork and also\naccording to the plan, but I rastroilo that the endometrium 6mm the doctor said a little small and\nappointed the main with 11 days to 25 Who drank it as well And it does I\njust where I read that \n\n']},
 {'code': 200,
  'lang': 'ru-en',
  'text': ['Girls hi I Want to sell all the drugs from IVF \n\n']},
 {'code': 200,
  'lang': 'ru-en',
  'text': ['Girls are in the hospital with terrible morning sickness the last few days feel\nbetter antiseptic before drinking 2 pills a day the last 3 day one \ntoday, the doctor said to cancel and I only have 14 weeks what do you think\nCancel \n\n']},
 {'code': 200,
  'lang': 'ru-en',
  'text': ["Hello Time 13 14 weeks yesterday there was spotting brown after\nsexual intercourse and the raising of the child Went for an ultrasound a beating heart, all\nthe order said that raised the children's place and therefore misses In\